In [1]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

# STATION 

Description: This script takes in a file named Station.csv(default) , recognizes its information and interprets the columns. Finally prints and save a Dat format file with the predefined tables for further processing

Station Data Conversion REFERENCE : 

ORDER : Padded with incremental index numbering

Key : Column "Name" in Station.csv  . Erased blank spaces.('Example_ _')

Name : Column "Desc" in Station.csv . Erased blank spaces.('Example_ _')

AOR : Column Zones in Station.csv. 

The .str.rstrip() method is widely used since many fields (almost all of them) have their value ending with two blank spaces.

In [2]:
#Input Filename (default = Station.csv)
InputStationFile = "Station.csv"

In [3]:
df = pd.read_csv(InputStationFile, skipinitialspace=True)
df.columns = df.columns.str.strip()

df_output = pd.DataFrame()
df_output['Order'] = range(1, len(df)+1) # Order column was not  specified in documentation. So in default is a unique incremental index.
df_output['Key'] = '"'+ df['Name'].str.rstrip() + '"'  
df_output['Name'] = '"' + df['Desc'].str.rstrip() + '"' 
df_output['AOR'] = df['Zones']

#sorting
df_output = df_output[['Order', 'Key', 'Name', 'AOR']]
df_station = df_output

#verification
print(df_station)

    Order     Key                        Name                AOR
0       1    "DI"     "DR INLET DISTRIBUTION"                3  
1       2    "BY"      "BRICKYARD SUBSTATION"                4  
2       3    "WS"  "WORTHINGTON SPRINGS DIST"                5  
3       4    "WA"          "WALDO SUBSTATION"                1  
4       5    "AB"           "ASTOR SWITCHING"                1  
..    ...     ...                         ...                ...
68     69  "SYS$"       "SYTEM PSUEDO POINTS"  1 2 3 4 5 6 7 8  
69     70    "BL"          "BLAND SUBSTATION"                5  
70     71    "WO"    "WATER OAK DISTRIBUTION"                3  
71     72    "AS"        "ASTOR DISTRIBUTION"                7  
72     73  "ODA$"         "ODA PSEUDO ANALOG"  1 2 3 4 5 6 7 8  

[73 rows x 4 columns]


In [4]:
# Crear un diccionario que mapee los valores actuales a los valores de reemplazo
replacement_dict = {
    '1  ': 1,
    '2  ': 2,
    '3  ': 3,
    '4  ': 4,
    '5  ': 5,
    '6  ': 6,
    '7  ': 7,
    '8  ': 8,
    '1 2 3 4 5 6 7  ': 9,
    '1 2 3 4 5 6 7 8  ': 10,
    '2 8  ': 11,
    '7 8  ': 12,
    '1 2  ': 13,
    '1 4  ': 14,
    '3 4  ': 15,
    '3 8  ': 16
}

df_station['AOR'] = df_station['AOR'].replace(replacement_dict)


In [5]:
df_station

,Order,Key,Name,AOR
0,1,"""DI""","""DR INLET DISTRIBUTION""",3
1,2,"""BY""","""BRICKYARD SUBSTATION""",4
2,3,"""WS""","""WORTHINGTON SPRINGS DIST""",5
3,4,"""WA""","""WALDO SUBSTATION""",1
4,5,"""AB""","""ASTOR SWITCHING""",1
...,...,...,...,...
68,69,"""SYS$""","""SYTEM PSUEDO POINTS""",10
69,70,"""BL""","""BLAND SUBSTATION""",5
70,71,"""WO""","""WATER OAK DISTRIBUTION""",3
71,72,"""AS""","""ASTOR DISTRIBUTION""",7


# Select Output name: (default: station_dat.dat)

In [6]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

stationfilename = os.path.join(folder_name, "station_dat.dat")

now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
with open(stationfilename, 'w') as f:
    f.write("*\n")
    f.write("*\n")
    f.write(f"* Creation Date/Time: {now}\n")
    f.write("*\n")
    f.write("*\tOrder\tKey\tName\tAOR\n")
    f.write("*\n")
    f.write("\t2\tSTATION\t0\t3\t4\t13\n")
    f.write("*\n")

    for index, row in df_station.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format
                ("", row['Order'], row['Order'], row['Key'], row['Name'], row['AOR']))
    f.write(" 0")  

# END STATION

--------------------------------------------------

# DEVICE_INSTANCE

Description: This script takes in a file named STATUS.csv , uses a column named 'NAME'
Instruction : Remove the first three characters so only the text after the comma remains. Then delete repetitions . From the 5000 records we should end with about 3800. 

Desired output:

*/             Order  Name
*/             ----  -----      
	53	DEVICE_INSTANCE	0
*---------------------------------------------------------------
	1	"MTXTOT"
	2	"DRD"
	3	"35C109"  

In [7]:
df_device_instance = pd.read_csv('Status.csv')

In [8]:
df_device_instance = df_device_instance[['Name  ']]

In [9]:
df_device_instance['Name  '] = df_device_instance['Name  '].str.split(',', expand=True)[1].str.strip()

In [10]:
df_device_instance

,Name
0,MTXTOT
1,DRD
2,21CP13
3,S8W219
4,25YBSC
...,...
7979,NaN
7980,NaN
7981,NaN
7982,NaN


In [11]:
df_device_instance = df_device_instance.drop_duplicates()

In [12]:
df_device_instance

,Name
0,MTXTOT
1,DRD
2,21CP13
3,S8W219
4,25YBSC
...,...
7705,T35P20
7706,T35P21
7707,T35P23
7708,TU86B


In [13]:
df_device_instance['Number'] = range(1,len(df_device_instance)+1)

/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/4099964339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_device_instance['Number'] = range(1,len(df_device_instance)+1)


In [14]:
df_device_instance = df_device_instance[['Number', 'Name  ']]

In [15]:
df_device_instance

,Number,Name
0,1,MTXTOT
1,2,DRD
2,3,21CP13
3,4,S8W219
4,5,25YBSC
...,...,...
7705,3057,T35P20
7706,3058,T35P21
7707,3059,T35P23
7708,3060,TU86B


In [16]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [17]:

device_inst_filename = os.path.join(folder_name, "device_instance_dat.dat")

In [18]:
with open(device_inst_filename, 'w') as f:
    f.write("*\n")
    f.write("\t53\tDEVICE_INSTANCE\t0\n")
    f.write("*\tIndex\tName\n")

    for index, row in df_device_instance.iterrows():
        f.write("\t{}\t{}\n".format(row['Number'], row['Name  ']))
    
    f.write("0")


In [19]:
df_analog_instance = pd.read_csv('Analog.csv')

df_analog_instance = df_analog_instance[['Name  ']]
df_analog_instance['Name  '] = df_analog_instance['Name  '].apply(lambda x: x.split(',')[1].strip())

#df_device_instance['Name  '] = df_device_instance['Name  '].str.split(',', expand=True)[1].str.strip()

#df_analog_instance['Name  '] = df_analog_instance['Name  '].strsplit(',', expand=True)[1].str.strip()
df_analog_instance = df_analog_instance.drop_duplicates()


In [20]:
df_analog_instance

,Name
0,W543A
1,W543B
2,W543C
3,WNDSPD
4,RC
...,...
4266,6W131W
4267,6W266W
4288,HRTB10
4291,HRTB11


In [21]:

# Concatenar los DataFrames df_device_instance y df_analog_instance
result_df = pd.concat([df_device_instance, df_analog_instance], ignore_index=True)
result_df.drop_duplicates(subset='Name  ', inplace=True, keep='first')

# Mostrar el DataFrame resultante
print(result_df)


      Number  Name  
0        1.0  MTXTOT
1        2.0     DRD
2        3.0  21CP13
3        4.0  S8W219
4        5.0  25YBSC
...      ...     ...
4216     NaN  6W131W
4217     NaN  6W266W
4218     NaN  HRTB10
4219     NaN  HRTB11
4220     NaN  P69CHT

[4219 rows x 2 columns]


In [22]:
result_df

,Number,Name
0,1.0,MTXTOT
1,2.0,DRD
2,3.0,21CP13
3,4.0,S8W219
4,5.0,25YBSC
...,...,...
4216,NaN,6W131W
4217,NaN,6W266W
4218,NaN,HRTB10
4219,NaN,HRTB11


In [23]:
df_device_instance = result_df

In [24]:
df_device_instance['Number'] = range(1,len(df_device_instance)+1)

In [25]:
df_device_instance

,Number,Name
0,1,MTXTOT
1,2,DRD
2,3,21CP13
3,4,S8W219
4,5,25YBSC
...,...,...
4216,4215,6W131W
4217,4216,6W266W
4218,4217,HRTB10
4219,4218,HRTB11


In [26]:
device_inst_filename = os.path.join(folder_name, "device_instance_dat.dat")

In [27]:
with open(device_inst_filename, 'w') as f:
    f.write("*\n")
    f.write("\t53\tDEVICE_INSTANCE\t0\n")
    f.write("*\tIndex\tName\n")

    for index, row in df_device_instance.iterrows():
        f.write("\t{}\t{}\n".format(row['Number'], row['Name  ']))
    
    f.write("0")


_________________

_________

# STATUS

Description: This script takes in a file named Status.csv(default) , recognizes its information and interprets the columns. Finally prints and save a Dat format file with the predefined tables for further processing

Station Data Conversion REFERENCE : 

(1)Type : ( 
       
            If column "Telem_A" in status.csv = empty. C_IND.   Type = 5

            If column "Telem_A" in status.csv = any number. T_IND.  Type = 1
            
            If column "Open_B" in status.csv = 12.  T_I&C.     Type = 2  ) 
                Criteria: only 42 rows (0,49%) has Open_B= 12.
                        so its gonna be Type = 2 regardless the value of Telem_A, 
                        giving priority to Open_B.
                        (In evey case of Open_B=12, Telem_A showed a number also)
              
              Edited:  IF column Stn = 054 (PS. PSEUDO POINTS) , M_IND. Type = 8 


----------
 15.1 Updates for TYPE: (
       
       "RG 1/15:
        If (PreSuffix == 7) then (T_R&L) Type=6 
        If (PreSuffix ==20 and (Desc contains 'LTC' or Desc contains 'REG')) then (T_CTL) Type=3 
Telem_A=any number then (T_IND) Type =1
Open_B=12 then (T_I&C) Type =2
Telem_A=<blank>  then (C_IND) Type =5
"

 )

--------
(3)Key : Format XXYYYZZZ
                         XX =   If type (descripted before) = 1 , XX = 01
                                If type (descripted before) = 2 , XX = 01
                                If type (descripted before) = 5 , XX = 02
                                If type (descripted before) = 8 , XX = 12
                         YYY = Stn (Station Order number.)(descripted in this Markdown)
                         ZZZ = Incremental number per station

(4)Name : Column "Name" + column "Desc" in Status.csv. Replace the commas (,) with blank spaces.

      *****
       Updated : 2019-03-06 . (4) Name : Column "Desc" in Status.csv . If not defined will be set to Key 

(5) Stn : (XXX) -> The first characters of the "Name" column in the current dataset, before the comma (,), reference the value of the KEY column in the previous dataset (Station). This value is based on the Order column. This number should be expressed in three digits (e.g., 38 = 038).

(10) Aor : Column "Zones" in Status.csv

(19) pState : Column "Presuffx" in Status.csv

(49) Norm : Column "Normal_State" in Status.csv

(29) AlarmGroup: Will be set to 1 unless defined in mapping document

(41) ICAddress : Refer to documentation. This script leaves it in blank
  **update 01/22 : ICAddress . double quotations "" 

* ***UPDATE : 
(107) pDeviceInstance : Column "Name" in Status.csv . Get the string after the comma, then search for the matching record of the objet DEVICE_INSTANCE. 


TYPE update
RG 1/18: If (Open_A is.notEmpty && Close_A is.NotEmpty) then (T_I&C) Type =2

UPDATE : 3/26/24
(44) pCtrlState : If point type is 6 (T_R/L) then set to 238 , else leave blank "".
(74) pScale :  If point type is 6 (T_R/L) then set to 25, else leave blank "".

In [28]:
#Input filename(default: "Status.csv")
InputStatusFile = "Status.csv"

In [29]:
status_df = pd.read_csv(InputStatusFile)
status_df.dropna(how='all', inplace=True)
df_status = pd.DataFrame()

In [30]:
status_df['PreSuffx  ']

0        8.0
1        5.0
2       26.0
3       15.0
4       26.0
        ... 
7705    26.0
7706    26.0
7707    26.0
7708    14.0
7709    26.0
Name: PreSuffx  , Length: 7710, dtype: float64

In [31]:
df_status['record'] = range(1, len(status_df)+1) 
df_status['OrderNo'] = range(1, len(status_df)+1) 
##### TYPE ######
status_df['PreSuffx  '] = status_df['PreSuffx  '].astype(str).str.strip()

#### conditions ###
cond1 = status_df['PreSuffx  '].str.contains('20') & status_df['Desc  '].str.contains('LTC|REG')
cond2 = status_df['PreSuffx  '].isin(['7.0', '7'])
cond3 = (status_df['Open_A  '] != '  ') & (status_df['Close_A  '] != '  ')
cond4 = (status_df['Open_B  '] == '12  ')
cond5 = (status_df['Telem_A  '].replace('  ', np.nan).notna())
cond6 = np.bitwise_not(status_df['Telem_A  '].replace('  ', np.nan).notna()) 


## values 
conditions = [cond1, cond2, cond3, cond4, cond5, cond6]
values = [3, 6, 2, 2, 1, 5]
#conditions.append(cond5)  
#values.append(2)

df_status['Type'] = np.select(conditions, values, default=5)

#df_status['Type'] = np.where(status_df['Open_B  '] == '12  ', 2, np.where(status_df['Telem_A  '].replace('  ', np.nan).notna(), 1, 5))
####################
df_status['Name'] = status_df['Desc  ']
df_status['AOR'] = status_df['Zones  '].str.rstrip()
df_status['AOR'] = df_status['AOR'].str.replace(r'\s+', '', regex=True)
df_status['pState'] = status_df['PreSuffx  ']
df_status['Norm'] = status_df['Normal_State  ']
df_status['AlarmGroup'] = 1
df_status['ICAddress'] = '""'


In [32]:
df_status['Type'].value_counts()

Type
1    4591
5    1751
2    1175
3      97
6      96
Name: count, dtype: int64

_______

In [33]:
status_df['PreSuffx  '].unique()

array(['8.0', '5.0', '26.0', '15.0', '25.0', '13.0', '19.0', '10.0',
       '1.0', '4.0', '21.0', '20.0', '9.0', '17.0', '3.0', '7.0', '2.0',
       '14.0', '28.0', '30.0', '16.0', '33.0', '22.0', '23.0', '0.0',
       '12.0', '29.0', '18.0', '31.0', '32.0'], dtype=object)

________

In [34]:

replacement_dict = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '1234567': 9,
    '12345678': 10,
    '28': 11,
    '78': 12,
    '12': 13,
    '14': 14,
    '34': 15,
    '38': 16
}


df_status['AOR'] = df_status['AOR'].replace(replacement_dict)


In [35]:
df_status

,record,OrderNo,Type,Name,AOR,pState,Norm,AlarmGroup,ICAddress
0,1,1,1,MOBIL TX HOT OIL TEMP,4,8.0,0.0,1,""""""
1,2,2,1,PH DIFF RELAY DISABLE,5,5.0,0.0,1,""""""
2,3,3,1,BW 2100 CPORT 13 SPARE,4,26.0,0.0,1,""""""
3,4,4,5,SB PSEUDO BREAKER,1,15.0,0.0,1,""""""
4,5,5,1,YARD BOX 2505 SELF CHECK,5,26.0,0.0,1,""""""
...,...,...,...,...,...,...,...,...,...
7705,7706,7706,1,TU 421-2 CP20 (MBIT),3,26.0,0.0,1,""""""
7706,7707,7707,1,TU 311C-2 CP21 (MBIT),3,26.0,0.0,1,""""""
7707,7708,7708,1,TU 487B CP23 (MBIT),3,26.0,0.0,1,""""""
7708,7709,7709,1,TU BUS LOCK-OUT RELAY,3,14.0,0.0,1,""""""


In [36]:
status_df

,Name,Desc,Zones,PType,DevClass,PreSuffx,Telem_CL,Telem_RTU,Telem_A,Telem_B,...,Vcode_State0,Vcode_State1,Vcode_State2,Vcode_State3,Normal_State,Dual_Normal,Alarm_Task,Annc_Delay,Alarm_Fmt,Meter_PT
0,"OP,MTXTOT",MOBIL TX HOT OIL TEMP,4,18.0,2.0,8.0,4,23,8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,120.0,130.0,
1,"PH,DRD",PH DIFF RELAY DISABLE,5,22.0,2.0,5.0,5,5,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,130.0,
2,"BW,21CP13",BW 2100 CPORT 13 SPARE,4,22.0,2.0,26.0,4,25,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,120.0,130.0,
3,"PS,S8W219",SB PSEUDO BREAKER,1,32.0,3.0,15.0,,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,0.0,
4,"PH,25YBSC",YARD BOX 2505 SELF CHECK,5,22.0,2.0,26.0,5,5,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,120.0,130.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,"LC,T35P20",TU 421-2 CP20 (MBIT),3,22.0,2.0,26.0,3,5,14,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,120.0,130.0,
7706,"LC,T35P21",TU 311C-2 CP21 (MBIT),3,22.0,2.0,26.0,3,5,14,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,120.0,130.0,
7707,"LC,T35P23",TU 487B CP23 (MBIT),3,22.0,2.0,26.0,3,5,14,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,120.0,130.0,
7708,"LC,TU86B",TU BUS LOCK-OUT RELAY,3,34.0,2.0,14.0,3,5,11,0,...,0.0,0.0,0.0,0.0,0.0,0.0,,0.0,131.0,


In [37]:

#df_status['TempName'] = status_df['Name  '].str[3:]

# Elimina espacios en blanco al final de la columna temporal
#df_status['TempName'] = df_status['TempName'].str.rstrip()



df_status['TempName'] = status_df['Name  '].str.split(',', expand=True)[1].str.strip()

# Realiza la asignación de números basada en el nombre temporal
df_status['pDeviceInstance'] = df_status['TempName'].map(df_device_instance.set_index('Name  ')['Number'])

# Borra la columna temporal
df_status.drop(columns=['TempName'], inplace=True)

# Muestra el DataFrame resultante con la nueva columna
print(df_status)



      record  OrderNo  Type                        Name  AOR pState  Norm  \
0          1        1     1     MOBIL TX HOT OIL TEMP      4    8.0   0.0   
1          2        2     1     PH DIFF RELAY DISABLE      5    5.0   0.0   
2          3        3     1    BW 2100 CPORT 13 SPARE      4   26.0   0.0   
3          4        4     5         SB PSEUDO BREAKER      1   15.0   0.0   
4          5        5     1  YARD BOX 2505 SELF CHECK      5   26.0   0.0   
...      ...      ...   ...                         ...  ...    ...   ...   
7705    7706     7706     1     TU 421-2 CP20  (MBIT)      3   26.0   0.0   
7706    7707     7707     1     TU 311C-2 CP21 (MBIT)      3   26.0   0.0   
7707    7708     7708     1       TU 487B CP23 (MBIT)      3   26.0   0.0   
7708    7709     7709     1     TU BUS LOCK-OUT RELAY      3   14.0   0.0   
7709    7710     7710     1    FI 2032-3 TIE BKR2 CP4      4   26.0   0.0   

      AlarmGroup ICAddress  pDeviceInstance  
0              1        ""   

# Stn :

This code strips the Name column in status and search the matching row Key in Station Dataframe
then, it reads the Order column value and assign this to the Stn column

In [38]:
status_df['Key'] = status_df['Name  '].str.split(',').str[0].str.strip()  # make sure no blank spaces are in the begin & end of the name
df_station['Key'] = df_station['Key'].str.replace('"', '')
stn_values = []  #list for store stn values

In [39]:
status_df['Key'] = status_df['Name  '].str.split(',').str[0].str.strip()

# Lista para almacenar los valores de 'Stn'
stn_values = []

for i in range(len(status_df)):
    key_value = status_df.loc[i, 'Key']
    
    if key_value == 'PS':
        # Extrae los dos primeros caracteres después de la coma
        first_two_chars_after_comma = status_df.loc[i, 'Name  '].split(',')[1][:2].strip()

        # Verifica si first_two_chars_after_comma está en df_station['Key']
        if first_two_chars_after_comma in df_station['Key'].values:
            key_value = first_two_chars_after_comma
        elif first_two_chars_after_comma == 'FP':  # Nueva condición para FP
            print(f"FP encontrado, cambiando a FSP en fila {i}")
            key_value = 'FSP'
        elif first_two_chars_after_comma == 'UC':  # Nueva condición para UC
            print(f"UC encontrado, cambiando a UCI en fila {i}")
            key_value = 'UCI'
        else:
            key_value = 'PS'
    
    matching_row = df_station[df_station['Key'] == key_value]
    if not matching_row.empty:
        stn_values.append(f"{matching_row['Order'].values[0]:03}")
    else:
        stn_values.append(np.nan)

df_status['Stn'] = stn_values


In [40]:
count_054 = df_status['Stn'].value_counts().get('055', 0)
print(f"El conteo de '055' (PS) en la columna 'Stn' es: {count_054}")


El conteo de '055' (PS) en la columna 'Stn' es: 187


In [41]:
df_status['Stn'].value_counts()

Stn
014    304
019    294
031    271
001    235
038    228
      ... 
053      4
048      3
073      3
044      2
069      1
Name: count, Length: 68, dtype: int64

In [42]:
df_status.head(20)

,record,OrderNo,Type,Name,AOR,pState,Norm,AlarmGroup,ICAddress,pDeviceInstance,Stn
0,1,1,1,MOBIL TX HOT OIL TEMP,4,8.0,0.0,1,"""""",1,016
1,2,2,1,PH DIFF RELAY DISABLE,5,5.0,0.0,1,"""""",2,025
2,3,3,1,BW 2100 CPORT 13 SPARE,4,26.0,0.0,1,"""""",3,015
3,4,4,5,SB PSEUDO BREAKER,1,15.0,0.0,1,"""""",4,055
4,5,5,1,YARD BOX 2505 SELF CHECK,5,26.0,0.0,1,"""""",5,025
5,6,6,5,MI FAST SCAN,3,25.0,0.0,1,"""""",6,011
6,7,7,1,BKR 4 LOC/REM SW,3,13.0,0.0,1,"""""",7,011
7,8,8,1,PH BKR 3 LOC/REM SW,5,13.0,0.0,1,"""""",8,025
8,9,9,5,GREEN COVE FAST SCAN,4,25.0,0.0,1,"""""",6,022
9,10,10,5,BC FAST SCAN,4,25.0,0.0,1,"""""",6,007


Replacement case: Pseudo Points.
IF stn = 054 (PS. Pseudo points.) Set type to 8 .   And then set XX Value to 12

In [43]:
df_status

,record,OrderNo,Type,Name,AOR,pState,Norm,AlarmGroup,ICAddress,pDeviceInstance,Stn
0,1,1,1,MOBIL TX HOT OIL TEMP,4,8.0,0.0,1,"""""",1,016
1,2,2,1,PH DIFF RELAY DISABLE,5,5.0,0.0,1,"""""",2,025
2,3,3,1,BW 2100 CPORT 13 SPARE,4,26.0,0.0,1,"""""",3,015
3,4,4,5,SB PSEUDO BREAKER,1,15.0,0.0,1,"""""",4,055
4,5,5,1,YARD BOX 2505 SELF CHECK,5,26.0,0.0,1,"""""",5,025
...,...,...,...,...,...,...,...,...,...,...,...
7705,7706,7706,1,TU 421-2 CP20 (MBIT),3,26.0,0.0,1,"""""",3057,023
7706,7707,7707,1,TU 311C-2 CP21 (MBIT),3,26.0,0.0,1,"""""",3058,023
7707,7708,7708,1,TU 487B CP23 (MBIT),3,26.0,0.0,1,"""""",3059,023
7708,7709,7709,1,TU BUS LOCK-OUT RELAY,3,14.0,0.0,1,"""""",3060,023


In [44]:
#mask = df_status['Name'].str.startswith("PS")
#mask = mask.fillna(False)
#df_status.loc[mask, 'Type'] = 8
df_status.loc[df_status['Name'].str.startswith("PS"), 'Type'] = 8

for i in range(len(df_status)):
    if df_status.loc[i, 'Stn'] == '054':
        key_to_search = df_status.loc[i, 'Name'][3:5]

        # Buscar valor 
        matching_row = df_station[df_station['Key'] == key_to_search]

        # != una fila correspondiente, 'Stn' en 'df_status'
        if not matching_row.empty:
            df_status.loc[i, 'Stn'] = f"{matching_row['Order'].values[0]:03}"

# KEY

In [45]:
key_values = []
xx_yyy_counters = {}  # creating a dict to store and count every YYY. It's used for a incremental ZZZ

In [46]:
df_status['Type'].value_counts()

Type
1    4591
8    1314
2    1175
5     437
3      97
6      96
Name: count, dtype: int64

In [47]:
for i in range(len(df_status)):
    
    if df_status.loc[i, 'Type'] == 2:
        xx = '01'
    elif df_status.loc[i, 'Type'] == 3:
        xx = '41'
    elif df_status.loc[i, 'Type'] == 6:
        xx = '41'
    elif df_status.loc[i, 'Type'] == 1:
        xx = '01'
    elif df_status.loc[i, 'Type'] == 5:
        xx = '02'
    elif df_status.loc[i, 'Type'] == 8:
        xx = '12'
    else:
        xx = '99'  # ERROR CASE . In case of being unable to find a coincidence.

    yyy = df_status.loc[i, 'Stn']

    #print(f'xx= {xx}  yyy= {yyy}  ')
    # combination
    key = xx + yyy

    # if key exists in counters, then add +1 , else, initialites it.
    if key in xx_yyy_counters:
        xx_yyy_counters[key] += 1
    else:
        xx_yyy_counters[key] = 1

    #   Current Value of ZZZ . 
    zz = f"{xx_yyy_counters[key]:03}"  #03 (Default). the number of Z in the format (default XX YYY ZZZ (3 Z))

    key_values.append(xx + yyy + zz)


In [48]:
df_status['Key'] = key_values

In [49]:
#obtaining and showing the counting of Types.
type_counts = df_status['Type'].value_counts()
print(type_counts)

Type
1    4591
8    1314
2    1175
5     437
3      97
6      96
Name: count, dtype: int64


In [50]:
df_status

,record,OrderNo,Type,Name,AOR,pState,Norm,AlarmGroup,ICAddress,pDeviceInstance,Stn,Key
0,1,1,1,MOBIL TX HOT OIL TEMP,4,8.0,0.0,1,"""""",1,016,01016001
1,2,2,1,PH DIFF RELAY DISABLE,5,5.0,0.0,1,"""""",2,025,01025001
2,3,3,1,BW 2100 CPORT 13 SPARE,4,26.0,0.0,1,"""""",3,015,01015001
3,4,4,5,SB PSEUDO BREAKER,1,15.0,0.0,1,"""""",4,055,02055001
4,5,5,1,YARD BOX 2505 SELF CHECK,5,26.0,0.0,1,"""""",5,025,01025002
...,...,...,...,...,...,...,...,...,...,...,...,...
7705,7706,7706,1,TU 421-2 CP20 (MBIT),3,26.0,0.0,1,"""""",3057,023,01023209
7706,7707,7707,1,TU 311C-2 CP21 (MBIT),3,26.0,0.0,1,"""""",3058,023,01023210
7707,7708,7708,1,TU 487B CP23 (MBIT),3,26.0,0.0,1,"""""",3059,023,01023211
7708,7709,7709,1,TU BUS LOCK-OUT RELAY,3,14.0,0.0,1,"""""",3060,023,01023212


In [51]:
new_order = ['record', 'OrderNo','Type','Key','Name','Stn','AOR','pState','Norm','AlarmGroup','ICAddress','pDeviceInstance']
df_status = df_status[new_order]

In [52]:
df_status

,record,OrderNo,Type,Key,Name,Stn,AOR,pState,Norm,AlarmGroup,ICAddress,pDeviceInstance
0,1,1,1,01016001,MOBIL TX HOT OIL TEMP,016,4,8.0,0.0,1,"""""",1
1,2,2,1,01025001,PH DIFF RELAY DISABLE,025,5,5.0,0.0,1,"""""",2
2,3,3,1,01015001,BW 2100 CPORT 13 SPARE,015,4,26.0,0.0,1,"""""",3
3,4,4,5,02055001,SB PSEUDO BREAKER,055,1,15.0,0.0,1,"""""",4
4,5,5,1,01025002,YARD BOX 2505 SELF CHECK,025,5,26.0,0.0,1,"""""",5
...,...,...,...,...,...,...,...,...,...,...,...,...
7705,7706,7706,1,01023209,TU 421-2 CP20 (MBIT),023,3,26.0,0.0,1,"""""",3057
7706,7707,7707,1,01023210,TU 311C-2 CP21 (MBIT),023,3,26.0,0.0,1,"""""",3058
7707,7708,7708,1,01023211,TU 487B CP23 (MBIT),023,3,26.0,0.0,1,"""""",3059
7708,7709,7709,1,01023212,TU BUS LOCK-OUT RELAY,023,3,14.0,0.0,1,"""""",3060


In [53]:
df_status['Key'] = '"'+ df_status['Key'].str.rstrip() + '"'

In [54]:
df_status['Name'] = '"'+ df_status['Name'].str.rstrip() + '"'

In [55]:
df_status

,record,OrderNo,Type,Key,Name,Stn,AOR,pState,Norm,AlarmGroup,ICAddress,pDeviceInstance
0,1,1,1,"""01016001""","""MOBIL TX HOT OIL TEMP""",016,4,8.0,0.0,1,"""""",1
1,2,2,1,"""01025001""","""PH DIFF RELAY DISABLE""",025,5,5.0,0.0,1,"""""",2
2,3,3,1,"""01015001""","""BW 2100 CPORT 13 SPARE""",015,4,26.0,0.0,1,"""""",3
3,4,4,5,"""02055001""","""SB PSEUDO BREAKER""",055,1,15.0,0.0,1,"""""",4
4,5,5,1,"""01025002""","""YARD BOX 2505 SELF CHECK""",025,5,26.0,0.0,1,"""""",5
...,...,...,...,...,...,...,...,...,...,...,...,...
7705,7706,7706,1,"""01023209""","""TU 421-2 CP20 (MBIT)""",023,3,26.0,0.0,1,"""""",3057
7706,7707,7707,1,"""01023210""","""TU 311C-2 CP21 (MBIT)""",023,3,26.0,0.0,1,"""""",3058
7707,7708,7708,1,"""01023211""","""TU 487B CP23 (MBIT)""",023,3,26.0,0.0,1,"""""",3059
7708,7709,7709,1,"""01023212""","""TU BUS LOCK-OUT RELAY""",023,3,14.0,0.0,1,"""""",3060


In [56]:
#df_status['pState'] = df_status['pState'].astype(int)
#df_status['pState'] = df_status['pState'] + 201

df_status['pState'] = pd.to_numeric(df_status['pState'], errors='coerce').astype(int)
df_status['pState'] = df_status['pState'] + 201

In [57]:
def set_pctrlstate(row):
    if row['Type'] == 6:
        return 238
    else:
        return '""'

df_status['pCtrlState'] = df_status.apply(set_pctrlstate, axis=1)


In [58]:
def set_pscale(row):
    if row['Type'] == 6:
        return 25
    else:
        return '""'
        
df_status['pScale'] = df_status.apply(set_pscale, axis=1)


In [59]:
df_status

,record,OrderNo,Type,Key,Name,Stn,AOR,pState,Norm,AlarmGroup,ICAddress,pDeviceInstance,pCtrlState,pScale
0,1,1,1,"""01016001""","""MOBIL TX HOT OIL TEMP""",016,4,209,0.0,1,"""""",1,"""""",""""""
1,2,2,1,"""01025001""","""PH DIFF RELAY DISABLE""",025,5,206,0.0,1,"""""",2,"""""",""""""
2,3,3,1,"""01015001""","""BW 2100 CPORT 13 SPARE""",015,4,227,0.0,1,"""""",3,"""""",""""""
3,4,4,5,"""02055001""","""SB PSEUDO BREAKER""",055,1,216,0.0,1,"""""",4,"""""",""""""
4,5,5,1,"""01025002""","""YARD BOX 2505 SELF CHECK""",025,5,227,0.0,1,"""""",5,"""""",""""""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705,7706,7706,1,"""01023209""","""TU 421-2 CP20 (MBIT)""",023,3,227,0.0,1,"""""",3057,"""""",""""""
7706,7707,7707,1,"""01023210""","""TU 311C-2 CP21 (MBIT)""",023,3,227,0.0,1,"""""",3058,"""""",""""""
7707,7708,7708,1,"""01023211""","""TU 487B CP23 (MBIT)""",023,3,227,0.0,1,"""""",3059,"""""",""""""
7708,7709,7709,1,"""01023212""","""TU BUS LOCK-OUT RELAY""",023,3,215,0.0,1,"""""",3060,"""""",""""""


# DATASET Status ready.

The program will create a csv version of the Status Dataframe edited and builded here . Intended to be mixed with the original datasource "Status.csv". The name for this csv is gonna be "Status_xref.csv"

Columnas utilizadas de Status.csv(Datasource):
'Telem_A  ','Open_B  ','Stn  ', 'PreSuffx  ', 'Name  ','Desc  ', 'Zones  ', 'Presuffx  ', 'Normal_State  ', 'Open_A  ', 'Close_A  ', 
Columnas del objeto construido como dataframe 'Status':
TODAS las columnas generadas 

Crear status_xref.csv 

In [60]:


columnas_seleccionadas = [
    'Telem_A  ','Open_B  ','PreSuffx  ', 'Name  ','Desc  ', 'Zones  ', 'Normal_State  ', 'Open_A  ', 'Close_A  '  
]


source_status_df = pd.read_csv('Status.csv', usecols=columnas_seleccionadas)


In [61]:
source_status_df

,Name,Desc,Zones,PreSuffx,Telem_A,Open_A,Open_B,Close_A,Normal_State
0,"OP,MTXTOT",MOBIL TX HOT OIL TEMP,4,8.0,8,,,,0.0
1,"PH,DRD",PH DIFF RELAY DISABLE,5,5.0,1,,,,0.0
2,"BW,21CP13",BW 2100 CPORT 13 SPARE,4,26.0,3,,,,0.0
3,"PS,S8W219",SB PSEUDO BREAKER,1,15.0,,,,,0.0
4,"PH,25YBSC",YARD BOX 2505 SELF CHECK,5,26.0,1,,,,0.0
...,...,...,...,...,...,...,...,...,...
7979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# OUTPUT TO DAT FILE :

In [62]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [63]:
output_status_name = 'status_dat.dat'

In [64]:
statusfilename = os.path.join(folder_name, output_status_name)

In [65]:
with open(statusfilename, 'w') as f:
    f.write('*\n')
    f.write('\t4\tSTATUS\t0\t1\t3\t4\t5\t10\t19\t49\t29\t41\t44\t74\t107\n')
    f.write('*\trecord\tOrderNo\tType\tKey\tName\tStn\tAOR\tpState\tNorm\tAlarmGroup\tICAddress\tpCtrlState\tpScale\tpDeviceInstance\n')

    for index, row in df_status.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format
                (row['record'], row['OrderNo'], row['Type'], row['Key'], row['Name'], row['Stn'], row['AOR'], row['pState'], row['Norm'], row['AlarmGroup'], row['ICAddress'],row['pCtrlState'], row['pScale'] ,row['pDeviceInstance']))
    f.write("0")  

# STATUS XREF CREATION

In [66]:

columnas_seleccionadas = [
    'Telem_A  ','Open_B  ','PreSuffx  ', 'Name  ','Desc  ', 'Zones  ', 'Normal_State  ', 'Open_A  ', 'Close_A  '  
]


source_status_df = pd.read_csv('Status.csv', usecols=columnas_seleccionadas)
source_status_df = source_status_df.rename(columns={'Name  ': 'Name_Status_Source  '})
source_status_df.reset_index(drop=True, inplace=True)
df_status.reset_index(drop=True, inplace=True)
combined_df = pd.concat([df_status, source_status_df], axis=1)
combined_df.to_csv('status_xref.csv', index=False)



# END STATUS

-------------------------------

# ANALOG

Description: This script takes in a file named Analog.csv(default) , recognizes its information and interprets the columns. Finally prints and save a Dat format file with the predefined tables for further processing

ANALOG Data conversion REFERENCE :

 Type (1) :  Columns : 
 ///
 
            IF Telem_B  !=  21  . T_ANLG  = type : 1

        IF Telem_RTU = blank .   C_ANLG =  Type : 2

        IF NOT Defined. Manual = Type: 3 

* CRITERIA: 100% of the Telem_B column is different from 21, while 40% of the Telem_RTU column is blank. To prioritize visibility of Telem_RTU in case of conflict, the priority will be given to C_ANLG (Type 2).
    ///


Key (3)   :  XX YYY ZZZ

Name (4)  :  Columns :  Desc . 
                If not defined, set to KEY 

Stn (5)   :  ###  -> The first characters of the "Name" column in the current dataset, before the comma (,), reference the value of the KEY column in the Station dataset. This value has a number based on the Order column. This number should be expressed in three digits (e.g., 38 = 038).

AOR (10)  :   Column Zones in Analog.csv

pScale (24): Column EU_HI in Analog.csv

AlarmGrp  (42)  :  Set to default: 1

ICAddress (66)  : 12/29 : Value : "". If this is Nan or 0 , it will raise an error.
______________________________________
NominalHiLimits (77,1)

NominalHiLimits (77,2)

NominalHiLimits (77,3)

NominalHiLimits (77,4):  column * Alm_unrHi * in Analog.csv , named in this df:  Nominal_HiLim

 *Edited:  NominalHiLimits (77,4): RENAMED TO HiLim[1]  (Rsnblty)

NominalLowLimits (78,1)

NominalLowLimits (78,2)

NominalLowLimits (78,3)

NominalLowLimits (78,4):  column Alm_unrLo in Analog.csv  , named in this df: Nominal_LoLim

 *Edited: RENAMED TO LoLim[1]   (Rsnblty)

 ADDED:NominalHiLimits (77,0): Column Alm_preHi in Analog.csv, named in this df: HiLim[0]    (High)

 ADDED:NominalLowLimits (78,0): Colum Alm_preLo in Analog.csv, named in this df: LoLim[0]    (Low)
 

 RG 9/14: There are three cases. 

1. Hi limits are present but Lo are empty

If Alm_preHi, Alm_emgHi, Alm_unrHi have values but  Alm_preLo, Alm_emgLo, Alm_unrLo  are empty then set 78,0 to -99995 , 78,1 to -99996, 78,4 to -99999  and follow cells H89 and H90

2. Lo limits are present but Hi are empty

If Alm_preLo, Alm_emgLo, Alm_unrLo have values but Alm_preHi, Alm_emgHi, Alm_unrHi are empty then set 77,0 to 99995 , 77,1 to 99996, 77,4 to 99999  and follow cells H91 and H92

3. Hi and Lo limits are present. 

Follow indications of column H89, H90, H91 and H92
Put the actual values presents.

4. Hi and Lo are empty. Set 77,4 to 999999 and 78,4 -999999 , set 77,1 and 77,2 and 78,1 and 78,2 to 0.


_____________________________________

NominalPairInactive (91) : RG 9/14: For cases 1,2 and 3 of cell I89 set to 0 the 91,1 and 91,2 and 91,3  , and set to 1 the  91,4 and 91,5 .

91,1 91,2 91,3 . 91,4 91,5 



New item:

RawCountFormat (54) : Assign "3"

    
   

In [67]:
#Data CSV Name entry
analog_file = "Analog.csv"

las condiciones para el llenado de las columnas son las siguientes :

1. Hi limits are present but Lo are empty:

If 'Alm_preHi  ', 'Alm_emgHi  ', 'Alm_unrHi  '(columnas encontradas dentro de df_analog)  have values but  'Alm_preLo  ', 'Alm_emgLo  ', 'Alm_unrLo  '(columnas encontradas dentro de df_analog)  are empty , then set Alm_preLo (de df_new) to -99995 , 78,1(crear esta columna) to -99996, 78,4(crear esta columna) to -99999  

Condicion 2:
2. Lo limits are present but Hi are empty

If 'Alm_preLo  ', 'Alm_emgLo  ', 'Alm_unrLo  '(columnas de df_analog)  have values but 'Alm_preHi  ', 'Alm_emgHi  ', 'Alm_unrHi  ' are empty then set Alm_preHi(de df_new) to 99995 , 77,1(crear esta columna) to 99996, 77,4(crear esta columna ) to 99999 

Condicion 3:
3. Hi and Lo limits are present. 

Tomar los valores presentes en df_analog para las columnas 'Alm_preHi  ', 'Alm_unrHi  ' y 'Alm_preLo  ', 'Alm_unrLo  ' y asignarlos a las columnas 'Alm_preHi', 'Alm_unrHi' y 'Alm_preLo', 'Alm_unrLo' de df_new.




In [68]:
df_analog = pd.read_csv(analog_file)

df_new = pd.DataFrame()
df_new['record'] = range(1, len(df_analog)+1) 
df_new['OrderNo'] = range(1, len(df_analog)+1) 


df_new['Type'] = 1

count_type_3 = 0

for index, row in df_analog.iterrows():
    name = row['Name  ']
    telem_rtu = row['Telem_RTU  ']

    # Primera condición: si 'Name' contiene 'PS' en los primeros 4 caracteres
    if 'PS' in name[:4]:
        df_new.at[index, 'Type'] = 3
        count_type_3 += 1
        continue  # Saltar a la siguiente iteración

    # Segunda condición: si 'Telem_RTU' es igual a dos espacios en blanco
    if telem_rtu == "  ":
        df_new.at[index, 'Type'] = 2

# Las demás columnas

df_new['Name'] = df_analog['Desc  ']
df_new['AOR'] = df_analog['Zones  ']
df_new['AlarmGrp'] = 1
df_new['ICAddress'] = ''

print(f"'Type' = 3: {count_type_3}")

'Type' = 3: 32


HI LOW LIMITS

In [69]:

for col in ['77,0', '77,1', '77,2', '77,3', '77,4', '78,0', '78,1', '78,2', '78,3', '78,4']:
    df_new[col] = 0  

for col in ['91,1', '91,2', '91,3', '91,4', '91,5']:
    df_new[col] = np.nan


    # SE PRUEBA LO SIGUIENTE:
# Mapeo entre las columnas en df_analog y las columnas en df_new
column_mapping = {
    'Alm_emgHi  ': '77,1',
    'Alm_preHi  ': '77,0',
    'Alm_unrHi  ': '77,4',
    'Alm_preLo  ': '78,0',
    'Alm_emgLo  ': '78,1',
    'Alm_unrLo  ': '78,4',
}

# Aplicar las condiciones y asignar los valores
for analog_col, new_col in column_mapping.items():
    # Crear una máscara para identificar las filas donde el valor no es "0  " o es no nulo
    mask = (df_analog[analog_col] != "0  ") & (df_analog[analog_col].notna())
    # Asignar el valor de la columna en df_analog a la columna correspondiente en df_new
    df_new.loc[mask, new_col] = df_analog.loc[mask, analog_col]



    #####

# Condición 1:
mask1 = (
    df_analog[['Alm_preHi  ', 'Alm_emgHi  ', 'Alm_unrHi  ']].notna().any(axis=1) &
    df_analog[['Alm_preLo  ', 'Alm_emgLo  ', 'Alm_unrLo  ']].isin(["0  ", "  "]).all(axis=1)
)
df_new.loc[mask1, '78,0'] = -999995
df_new.loc[mask1, '78,1'] = -999996
df_new.loc[mask1, '78,4'] = -999999
df_new.loc[mask1, ['91,1', '91,2', '91,3']] = 0
df_new.loc[mask1, ['91,4', '91,5']] = 1


# Condición 2:
mask2 = (
    df_analog[['Alm_preLo  ', 'Alm_emgLo  ', 'Alm_unrLo  ']].notna().any(axis=1) &
    df_analog[['Alm_preHi  ', 'Alm_emgHi  ', 'Alm_unrHi  ']].isin(["0  ", "  "]).all(axis=1)
)
df_new.loc[mask2, '77,0'] = 999995
df_new.loc[mask2, '77,1'] = 999996
df_new.loc[mask2, '77,4'] = 999999
df_new.loc[mask2, ['91,1', '91,2', '91,3']] = 0
df_new.loc[mask2, ['91,4', '91,5']] = 1


# Condición 3:
df_new['77,0'] = df_analog['Alm_preHi  ']
df_new['77,4'] = df_analog['Alm_unrHi  ']
df_new['78,1'] = df_analog['Alm_emgLo  ']
df_new['78,4'] = df_analog['Alm_unrLo  ']
df_new.loc[df_new['77,0'].notna() | df_new['78,1'].notna(), ['91,1', '91,2', '91,3']] = 0
df_new.loc[df_new['77,0'].notna() | df_new['78,1'].notna(), ['91,4', '91,5']] = 1


# Condición 4:
mask4 = (
    df_analog[['Alm_preHi  ', 'Alm_emgHi  ', 'Alm_unrHi  ', 'Alm_preLo  ', 'Alm_emgLo  ', 'Alm_unrLo  ']].isin(["0  ", "  "]).all(axis=1)
)
df_new.loc[mask4, '77,4'] = 999999
df_new.loc[mask4, '78,4'] = -999999
df_new.loc[mask4, ['77,1', '77,2', '78,1', '78,2']] = 0


/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/1918518988.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['450  ' '450  ' '450  ' ... '  ' '  ' '525  ']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_new.loc[mask, new_col] = df_analog.loc[mask, analog_col]
/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/1918518988.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['400  ' '400  ' '400  ' ... '  ' '1  ' '500  ']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_new.loc[mask, new_col] = df_analog.loc[mask, analog_col]
/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/1918518988.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['4

In [70]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,"77,0","77,1","77,2",...,"78,0","78,1","78,2","78,3","78,4","91,1","91,2","91,3","91,4","91,5"
0,1,1,1,AB W543 A AMPS BKR TRIP,7,1,,400,450,0,...,-10,-20,0,0,-30,0.0,0.0,0.0,1.0,1.0
1,2,2,1,AB W543 B AMPS BKR TRIP,7,1,,400,450,0,...,-10,-20,0,0,-30,0.0,0.0,0.0,1.0,1.0
2,3,3,1,AB W543 C AMPS BKR TRIP,7,1,,400,450,0,...,-10,-20,0,0,-30,0.0,0.0,0.0,1.0,1.0
3,4,4,1,SG WIND SPEED,7,1,,35,74,0,...,-999995,,0,0,,0.0,0.0,0.0,1.0,1.0
4,5,5,2,CHANNEL ONE RETRY COUNT,1,1,,,0,0,...,-999995,0,0,0,-999999,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,4349,4349,1,WS TX TANK OIL TEMP,5,1,,NaN,0,0,...,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4349,4350,4350,1,DC VOLTS,5,1,,NaN,0,0,...,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4350,4351,4351,1,WS TX AMBIENT TEMP,5,1,,NaN,0,0,...,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4351,4352,4352,1,WS FDR 1 WATTS,5,1,,NaN,0,0,...,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
#df_analog['EU_Hi  '] = df_analog['EU_Hi  '].apply(keep_decimal_precision)
df_new['pScale EU_Hi'] = df_analog['EU_Hi  ']

In [72]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,"77,0","77,1","77,2",...,"78,1","78,2","78,3","78,4","91,1","91,2","91,3","91,4","91,5",pScale EU_Hi
0,1,1,1,AB W543 A AMPS BKR TRIP,7,1,,400,450,0,...,-20,0,0,-30,0.0,0.0,0.0,1.0,1.0,1.0
1,2,2,1,AB W543 B AMPS BKR TRIP,7,1,,400,450,0,...,-20,0,0,-30,0.0,0.0,0.0,1.0,1.0,1.0
2,3,3,1,AB W543 C AMPS BKR TRIP,7,1,,400,450,0,...,-20,0,0,-30,0.0,0.0,0.0,1.0,1.0,1.0
3,4,4,1,SG WIND SPEED,7,1,,35,74,0,...,,0,0,,0.0,0.0,0.0,1.0,1.0,100.0
4,5,5,2,CHANNEL ONE RETRY COUNT,1,1,,,0,0,...,0,0,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,4349,4349,1,WS TX TANK OIL TEMP,5,1,,NaN,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4349,4350,4350,1,DC VOLTS,5,1,,NaN,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4350,4351,4351,1,WS TX AMBIENT TEMP,5,1,,NaN,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4351,4352,4352,1,WS FDR 1 WATTS,5,1,,NaN,0,0,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


# STN 

In [73]:
df_analog['Key'] = df_analog['Name  '].str.split(',').str[0].str.strip()

stn_values = []

for i in range(len(df_analog)):
    
    matching_row = df_station[df_station['Key'] == df_analog.loc[i, 'Key']]
    
    if not matching_row.empty:
        stn_values.append(f"{matching_row['Order'].values[0]:03}")
    else:
        stn_values.append(np.nan)
df_new['Stn'] = stn_values

In [74]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,"77,0","77,1","77,2",...,"78,2","78,3","78,4","91,1","91,2","91,3","91,4","91,5",pScale EU_Hi,Stn
0,1,1,1,AB W543 A AMPS BKR TRIP,7,1,,400,450,0,...,0,0,-30,0.0,0.0,0.0,1.0,1.0,1.0,005
1,2,2,1,AB W543 B AMPS BKR TRIP,7,1,,400,450,0,...,0,0,-30,0.0,0.0,0.0,1.0,1.0,1.0,005
2,3,3,1,AB W543 C AMPS BKR TRIP,7,1,,400,450,0,...,0,0,-30,0.0,0.0,0.0,1.0,1.0,1.0,005
3,4,4,1,SG WIND SPEED,7,1,,35,74,0,...,0,0,,0.0,0.0,0.0,1.0,1.0,100.0,008
4,5,5,2,CHANNEL ONE RETRY COUNT,1,1,,,0,0,...,0,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0,038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,4349,4349,1,WS TX TANK OIL TEMP,5,1,,NaN,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,003
4349,4350,4350,1,DC VOLTS,5,1,,NaN,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,010
4350,4351,4351,1,WS TX AMBIENT TEMP,5,1,,NaN,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,003
4351,4352,4352,1,WS FDR 1 WATTS,5,1,,NaN,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,003


# KEY

In [75]:
cols = ['Alm_unrHi', 'Alm_unrLo', 'Alm_preHi', 'Alm_preLo', '78,1', '78,4', '77,1', '77,4']
for col in cols:
    if col not in df_new.columns:
        df_new[col] = np.nan



In [76]:
key_values = []
xx_yyy_counters = {}

In [77]:
for i in range(len(df_new)):
    
    # Asignamos el valor correspondiente a 'XX' según el valor de 'Type'
    if df_new.loc[i, 'Type'] == 1:
        xx = '03'
    elif df_new.loc[i, 'Type'] == 2:
        xx = '04'
    else:
        xx = '99'  # ERROR CASE


    yyy = str(df_new.loc[i, 'Stn'])

    key = xx + yyy

    if key in xx_yyy_counters:
        xx_yyy_counters[key] += 1
    else:
        xx_yyy_counters[key] = 1

    zz = f"{xx_yyy_counters[key]:03}"

    key_values.append(xx + yyy + zz)

df_new['Key'] = key_values

In [78]:
df_new

,record,OrderNo,Type,Name,AOR,AlarmGrp,ICAddress,"77,0","77,1","77,2",...,"91,3","91,4","91,5",pScale EU_Hi,Stn,Alm_unrHi,Alm_unrLo,Alm_preHi,Alm_preLo,Key
0,1,1,1,AB W543 A AMPS BKR TRIP,7,1,,400,450,0,...,0.0,1.0,1.0,1.0,005,NaN,NaN,NaN,NaN,03005001
1,2,2,1,AB W543 B AMPS BKR TRIP,7,1,,400,450,0,...,0.0,1.0,1.0,1.0,005,NaN,NaN,NaN,NaN,03005002
2,3,3,1,AB W543 C AMPS BKR TRIP,7,1,,400,450,0,...,0.0,1.0,1.0,1.0,005,NaN,NaN,NaN,NaN,03005003
3,4,4,1,SG WIND SPEED,7,1,,35,74,0,...,0.0,1.0,1.0,100.0,008,NaN,NaN,NaN,NaN,03008001
4,5,5,2,CHANNEL ONE RETRY COUNT,1,1,,,0,0,...,0.0,1.0,1.0,1.0,038,NaN,NaN,NaN,NaN,04038001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,4349,4349,1,WS TX TANK OIL TEMP,5,1,,NaN,0,0,...,NaN,NaN,NaN,0.0,003,NaN,NaN,NaN,NaN,03003036
4349,4350,4350,1,DC VOLTS,5,1,,NaN,0,0,...,NaN,NaN,NaN,0.0,010,NaN,NaN,NaN,NaN,03010040
4350,4351,4351,1,WS TX AMBIENT TEMP,5,1,,NaN,0,0,...,NaN,NaN,NaN,0.0,003,NaN,NaN,NaN,NaN,03003037
4351,4352,4352,1,WS FDR 1 WATTS,5,1,,NaN,0,0,...,NaN,NaN,NaN,0.0,003,NaN,NaN,NaN,NaN,03003038


In [79]:
#df_new = df_new[['record', 'OrderNo','Type', 'Key', 'Name', 'Stn', 'AOR', 'Nominal_HiLim', 'Nominal_HiLim1', 'Nominal_LoLim', 'Nominal_LoLim1', 'pScale EU_Hi', 'AlarmGrp']].copy()
#df_new['ICAddress'] = "NaN"


df_ner['NominalPairInactive'] 

In [80]:

new_column_order = ['record', 'OrderNo', 'Type', 'Key', 'Name', 'Stn', 'AOR',
                    '77,0', '77,1', '77,2', '77,3', '77,4',
                    '78,0', '78,1', '78,2', '78,3', '78,4',
                    '91,1', '91,2', '91,3', '91,4', '91,5',
                    'pScale EU_Hi', 'AlarmGrp', 'ICAddress']


df_new = df_new[new_column_order]



In [81]:
df_new['77,1'].head(10)

0     450  
1     450  
2     450  
3      74  
4         0
5         0
6    999996
7         0
8         0
9         0
Name: 77,1, dtype: object

In [82]:
df_new['Key'] = '"'+ df_new['Key'].str.rstrip() + '"'
df_new['Name'] = '"'+ df_new['Name'].str.rstrip() + '"'

/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/820790713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Key'] = '"'+ df_new['Key'].str.rstrip() + '"'
/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/820790713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Name'] = '"'+ df_new['Name'].str.rstrip() + '"'


In [83]:
df_new.loc[df_new['77,4'] == 0, '77,4'] = 999999
df_new.loc[df_new['77,4'] == '  ', '77,4'] = 999999

df_new.loc[df_new['77,1'] == 0, '77,1'] = 999996
df_new.loc[df_new['77,1'] == '  ', '77,1'] = 999996

df_new.loc[df_new['77,0'] == 0, '77,0'] = 999995
df_new.loc[df_new['77,0'] == '  ', '77,0'] = 999995

#df_new.loc[df_new['78,0'] == 0, '78,0'] = -999995
df_new.loc[df_new['78,0'] == '  ', '78,0'] = -999995

df_new.loc[df_new['78,1'] == 0, '78,1'] = -999996
df_new.loc[df_new['78,1'] == '  ', '78,1'] = -999996

df_new.loc[df_new['78,4'] == 0, '78,4'] = -999999
df_new.loc[df_new['78,4'] == '  ', '78,4'] = -999999


In [84]:
borrador = """df_new['77,4'] = df_new['77,4'].replace({0: 999999, '  ': 999999})
df_new['77,1'] = df_new['77,1'].replace({0: 999996, '  ': 999996})
df_new['77,0'] = df_new['77,0'].replace({0: 999995, '  ': 999995})
df_new['78,0'] = df_new['78,0'].replace({0: -999995, '  ': -999995})
df_new['78,1'] = df_new['78,1'].replace({0: -999996, '  ': -999996})
df_new['78,4'] = df_new['78,4'].replace({0: -999999, '  ': -999999})"""


In [85]:
#df_analog['Alm_preLo  '].to_csv('borrrrar.csv')

In [86]:
df_analog['Alm_preLo  '] = df_analog['Alm_preLo  '].astype(str).str.strip()
df_new['78,0'] = df_new['78,0'].astype(str).str.strip()

# Encontrar las filas donde 'Alm_preLo' es '0' y la columna 78,0 es '-999995'
mask = (df_analog['Alm_preLo  '] == '0  ') & (df_new['78,0'] == '-999995')

# Copiar los valores de 'Alm_preLo' a la columna 78.0 en las filas correspondientes
df_new.loc[mask, '78,0'] = df_analog.loc[mask, 'Alm_preLo  ']




/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/1392638317.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['78,0'] = df_new['78,0'].astype(str).str.strip()


In [87]:
df_new.head(10)

,record,OrderNo,Type,Key,Name,Stn,AOR,"77,0","77,1","77,2",...,"78,3","78,4","91,1","91,2","91,3","91,4","91,5",pScale EU_Hi,AlarmGrp,ICAddress
0,1,1,1,"""03005001""","""AB W543 A AMPS BKR TRIP""",005,7,400,450,0,...,0,-30,0.0,0.0,0.0,1.0,1.0,1.0000,1,
1,2,2,1,"""03005002""","""AB W543 B AMPS BKR TRIP""",005,7,400,450,0,...,0,-30,0.0,0.0,0.0,1.0,1.0,1.0000,1,
2,3,3,1,"""03005003""","""AB W543 C AMPS BKR TRIP""",005,7,400,450,0,...,0,-30,0.0,0.0,0.0,1.0,1.0,1.0000,1,
3,4,4,1,"""03008001""","""SG WIND SPEED""",008,7,35,74,0,...,0,-999999,0.0,0.0,0.0,1.0,1.0,100.0000,1,
4,5,5,2,"""04038001""","""CHANNEL ONE RETRY COUNT""",038,1,999995,999996,0,...,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0000,1,
5,6,6,2,"""04038002""","""KH TX1 KVA AVERAGE""",038,1,999995,999996,0,...,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0000,1,
6,7,7,1,"""03017001""","""ALACHUA TX2 ACCUMULATOR""",017,5,999995,999996,0,...,0,-10,0.0,0.0,0.0,1.0,1.0,3.6234,1,
7,8,8,2,"""04023001""","""LC BRK 4 AMP TOTAL""",023,3,999995,999996,0,...,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0000,1,
8,9,9,2,"""04030001""","""CHANNEL SIX RETRY COUNT""",030,7,999995,999996,0,...,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0000,1,
9,10,10,2,"""04041001""","""ME BRK 1 AMP TOTAL""",041,1 2 3 4 5 6 7 8,999995,999996,0,...,0,-999999,0.0,0.0,0.0,1.0,1.0,1.0000,1,


In [88]:
df_new['AOR'] = df_new['AOR'].str.replace(r'\s+', '', regex=True)

/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/543210088.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['AOR'] = df_new['AOR'].str.replace(r'\s+', '', regex=True)


In [89]:
# Crear un diccionario que mapee los valores actuales a los valores de reemplazo
replacement_dict = {
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '1234567': 9,
    '12345678': 10,
    '28': 11,
    '78': 12,
    '12': 13,
    '14': 14,
    '34': 15,
    '38': 16
}

# Reemplazar los valores en la columna 'AOR' usando el diccionario de reemplazo
df_new['AOR'] = df_new['AOR'].replace(replacement_dict)

/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/2453514998.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['AOR'] = df_new['AOR'].replace(replacement_dict)


In [90]:
df_new['78,1'] = df_new['78,1'].replace({'0  ': -999996})
df_new['78,4'] = df_new['78,4'].replace({'0  ': -999999})


/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/1558125795.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['78,1'] = df_new['78,1'].replace({'0  ': -999996})
/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/1558125795.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['78,4'] = df_new['78,4'].replace({'0  ': -999999})


In [91]:
def modificar_type(fila):
    if fila['Name'].startswith('"PS'):
        return 3
    else:
        return fila['Type']
df_new['Type'] = df_new.apply(modificar_type, axis=1)


/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/659747593.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Type'] = df_new.apply(modificar_type, axis=1)


In [92]:
#df_new.to_csv('borrardf_new.csv', index=False)

In [93]:
filas_con_ps = df_new[df_new['Name'].str.startswith('"PS')]

# Contamos el número de filas
numero_de_filas = len(filas_con_ps)

print("Número de filas que comienzan con 'PS':", numero_de_filas)

Número de filas que comienzan con 'PS': 0


df_new [punit]

In [94]:


analog_file = "Analog.csv"
df_analog = pd.read_csv(analog_file)


df_unit = pd.DataFrame()
df_unit['name'] = df_analog['EuText  '].drop_duplicates(keep='first').reset_index(drop=True)
df_unit = df_unit[df_unit['name'].notna() & (df_unit['name'].str.strip() != '')]


df_unit['record'] = range(1, len(df_unit['name']) + 1)


unit_mapping = df_unit.set_index('name')['record'].to_dict()
df_new['pUNIT'] = df_analog['EuText  '].map(unit_mapping)
df_new['pUNIT'] = df_new['pUNIT'].apply(lambda x: '""' if pd.isna(x) else str(int(x)))

/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/2634846413.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['pUNIT'] = df_analog['EuText  '].map(unit_mapping)
/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/2634846413.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['pUNIT'] = df_new['pUNIT'].apply(lambda x: '""' if pd.isna(x) else str(int(x)))


In [95]:
df_new['pUNIT'].isna().sum()

0

In [96]:
empty_vals = df_new['pUNIT'].isna() | (df_new['pUNIT'] == '""')
num_empty = empty_vals.sum()
print(f'Number of empty values: {num_empty}')

Number of empty values: 150


In [97]:
df_new['pUNIT'].head(6)

0     1
1     1
2     1
3     2
4    ""
5     3
Name: pUNIT, dtype: object

In [98]:
#import csv 
#df_new.to_csv('deletethis.csv', escapechar='\\' ,quoting=csv.QUOTE_NONE)

In [99]:
df_new.loc[:, 'RawCountFormat'] = 3

/var/folders/mj/9v8yq1r92nqcqj3rqh0f2h2h0000gn/T/ipykernel_4960/3518185772.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[:, 'RawCountFormat'] = 3


In [100]:
df_new

,record,OrderNo,Type,Key,Name,Stn,AOR,"77,0","77,1","77,2",...,"91,1","91,2","91,3","91,4","91,5",pScale EU_Hi,AlarmGrp,ICAddress,pUNIT,RawCountFormat
0,1,1,1,"""03005001""","""AB W543 A AMPS BKR TRIP""",005,7,400,450,0,...,0.0,0.0,0.0,1.0,1.0,1.0,1,,1,3
1,2,2,1,"""03005002""","""AB W543 B AMPS BKR TRIP""",005,7,400,450,0,...,0.0,0.0,0.0,1.0,1.0,1.0,1,,1,3
2,3,3,1,"""03005003""","""AB W543 C AMPS BKR TRIP""",005,7,400,450,0,...,0.0,0.0,0.0,1.0,1.0,1.0,1,,1,3
3,4,4,1,"""03008001""","""SG WIND SPEED""",008,7,35,74,0,...,0.0,0.0,0.0,1.0,1.0,100.0,1,,2,3
4,5,5,2,"""04038001""","""CHANNEL ONE RETRY COUNT""",038,1,999995,999996,0,...,0.0,0.0,0.0,1.0,1.0,1.0,1,,"""""",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,4349,4349,1,"""03003036""","""WS TX TANK OIL TEMP""",003,5,NaN,999996,0,...,NaN,NaN,NaN,NaN,NaN,0.0,1,,13,3
4349,4350,4350,1,"""03010040""","""DC VOLTS""",010,5,NaN,999996,0,...,NaN,NaN,NaN,NaN,NaN,0.0,1,,5,3
4350,4351,4351,1,"""03003037""","""WS TX AMBIENT TEMP""",003,5,NaN,999996,0,...,NaN,NaN,NaN,NaN,NaN,0.0,1,,13,3
4351,4352,4352,1,"""03003038""","""WS FDR 1 WATTS""",003,5,NaN,999996,0,...,NaN,NaN,NaN,NaN,NaN,0.0,1,,3,3


Output Filename:

In [101]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [102]:
output_analog_name = 'analog_dat.dat'

In [103]:
analog_filename = os.path.join(folder_name, output_analog_name)

In [104]:
with open(analog_filename, 'w') as f:
    f.write('* \n')
    f.write('\t5\tANALOG\t0\t1\t3\t4\t5\t10\t23\t24\t42\t54\t77,4\t77,1\t77,0\t78,0\t78,1\t78,4\t91,1\t91,2\t91,3\t91,4\t91,5\t66\n')
    f.write('*\trecord\tOrderNo\tType\tKey\tName\tStn\tAOR\tpUNIT\tpScale EU_Hi\tAlarmGrp\tRawCountFormat\t77,4\t77,1\t77,0\t78,0\t78,1\t78,4\tNomPairInactive91,1\t91,2\t91,3\t91,4\t91,5\tICAddress\n')

    for index, row in df_new.iterrows():
        f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n".format
                ("", row['record'], row['OrderNo'], row['Type'], row['Key'], row['Name'], row['Stn'], row['AOR'],row['pUNIT'],row['pScale EU_Hi'], row['AlarmGrp'],row['RawCountFormat'], row['77,4'], row['77,1'], row['77,0'], row['78,0'], row['78,1'], row['78,4'], row['91,1'], row['91,2'], row['91,3'], row['91,4'], row['91,5'], row['ICAddress']))
    f.write("0")  

________

________

# ANALOG_CONFIG

vamos a crear un dataframe llamado Analog_config , el cual tendrá solo dos columnas, una columna se llamara Key , cuyo contenido será exactamente el contenido de Key del df  llamado df_new . y una columna se llamara name , que vendra de df_analog['Name  '] 

In [105]:

Analog_config = pd.DataFrame({
    'Key': df_new['Key'],
    'name': df_analog['Name  '].str.split(',', expand=True)[1].str.strip()# Obtener los caracteres a partir de la coma
    
})

#df_analog['Name  '].str.split(',', expand=True)[1].str.strip()


# Crear una función para buscar el número en df_device_instance
def find_number(name):
    match = df_device_instance[df_device_instance['Name  '] == name]
    if not match.empty:
        return match.iloc[0]['Number']
    else:
        return None

# Aplicar la función para obtener los números y almacenarlos en Analog_config['pDeviceInstance']
Analog_config['pDeviceInstance'] = Analog_config['name'].apply(find_number)

# Ahora, Analog_config contendrá la columna 'pDeviceInstance' con los números correspondientes.


In [106]:
Analog_config

,Key,name,pDeviceInstance
0,"""03005001""",W543A,3062
1,"""03005002""",W543B,3063
2,"""03005003""",W543C,3064
3,"""03008001""",WNDSPD,3065
4,"""04038001""",RC,3066
...,...,...,...
4348,"""03003036""",TXTEMP,3488
4349,"""03010040""",DCV,3090
4350,"""03003037""",TXABT,3280
4351,"""03003038""",1KW,3236


In [107]:
df_device_instance

,Number,Name
0,1,MTXTOT
1,2,DRD
2,3,21CP13
3,4,S8W219
4,5,25YBSC
...,...,...
4216,4215,6W131W
4217,4216,6W266W
4218,4217,HRTB10
4219,4218,HRTB11


In [108]:
print(set(Analog_config['name']).intersection(set(df_device_instance['Name  '])))


{'66', '03', 'W34KW', '116', '6W51AV', 'HRAVG', 'RSMV', '6W56AV', 'SE230A', 'RACSTP', 'LCTX2W', 'W34PF', 'L115PH', '9A', '67', 'L115PD', '6W54CW', 'G1KVA', 'PDY1', '3KW', '230CAL', 'W35VAR', 'PERCT', 'PFAVG', '87', 'XBCH', '6W57AA', '6164MV', 'LDEADB', '96', '114', 'SBMVT', '125', 'PF3AVG', 'WRNGRY', 'G4MV', 'ADD12', 'CHR32', 'TX2PMT', 'G7PFM', 'TIM3', 'W676B', 'CHR3', '24', 'MOPER', 'MVAVG', 'W1AVG', 'W36A', 'W543C', '29', 'RAKV', 'HOLD1', '6160MV', 'HRTBT6', 'W655W', 'T1HST1', 'CCDY', '4TOT', '143', 'TIM2', '146', '205', 'SINSPD', '2TOT', '181', '177', 'TX3TMP', 'TX1FRQ', 'TC3TMP', 'TC1TMP', 'PDYT', '20', 'RCTAP', 'TX1PMT', 'W655PF', 'TX2AMP', 'LQSHED', 'W3AVG', '3B', '1', 'BCNBV', 'KVAM', 'T2HST2', 'LC5B', 'PERCT4', 'PFM', 'CCALI', '6W56B', 'W36C', 'XPOLL', 'TX1ADD', '6KW', 'W676A', '26', 'PEAK', 'THST1', '69BAV', 'MAX', 'LTOTCH', 'HRTB11', '115CDY', 'TXMVT', '131', '152', 'WAVG', 'CL', 'TOTWAT', '192', 'PF3A', 'PER', 'W6208A', 'HOUR', '221', '224', 'HSBV', 'CDY2', 'W686B', '7ADD', 

In [109]:
print("Unique names in Analog_config:", Analog_config['name'].unique()[:10])  # muestra los primeros 10
print("Unique names in df_device_instance:", df_device_instance['Name  '].unique()[:10])  # muestra los primeros 10


Unique names in Analog_config: ['W543A' 'W543B' 'W543C' 'WNDSPD' 'RC' 'KV1AVG' 'P2' '4TOT' '1TOT' 'P']
Unique names in df_device_instance: ['MTXTOT' 'DRD' '21CP13' 'S8W219' '25YBSC' 'FS' '4LR' '3LR' '35CP21' '2BF']


In [110]:
df_device_instance

,Number,Name
0,1,MTXTOT
1,2,DRD
2,3,21CP13
3,4,S8W219
4,5,25YBSC
...,...,...
4216,4215,6W131W
4217,4216,6W266W
4218,4217,HRTB10
4219,4218,HRTB11


In [111]:
Analog_config

,Key,name,pDeviceInstance
0,"""03005001""",W543A,3062
1,"""03005002""",W543B,3063
2,"""03005003""",W543C,3064
3,"""03008001""",WNDSPD,3065
4,"""04038001""",RC,3066
...,...,...,...
4348,"""03003036""",TXTEMP,3488
4349,"""03010040""",DCV,3090
4350,"""03003037""",TXABT,3280
4351,"""03003038""",1KW,3236


In [112]:
# HABILITAR ESTO Analog_config.drop('name', axis=1, inplace = True )

In [113]:
Analog_config['record'] = range(1, len(Analog_config) + 1)


In [114]:
Analog_config

,Key,name,pDeviceInstance,record
0,"""03005001""",W543A,3062,1
1,"""03005002""",W543B,3063,2
2,"""03005003""",W543C,3064,3
3,"""03008001""",WNDSPD,3065,4
4,"""04038001""",RC,3066,5
...,...,...,...,...
4348,"""03003036""",TXTEMP,3488,4349
4349,"""03010040""",DCV,3090,4350
4350,"""03003037""",TXABT,3280,4351
4351,"""03003038""",1KW,3236,4352


In [115]:
Analog_config.to_csv('AnalogConfig.csv', index=False)

In [116]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)


In [117]:
analog_config_filename = "analog_config_dat.dat"


In [118]:
analog_config_path = os.path.join(folder_name, analog_config_filename)

In [119]:
with open(analog_config_path, 'w') as f:
    f.write("*\n")
    f.write("\t41\tANALOG_CONFIG\t0\t9\n")
    f.write("*\tKEY\tpDeviceInstance\n")

    for index, row in Analog_config.iterrows():
        f.write("\t{}\t{}\t{}\n".format(row['record'],row['Key'], row['pDeviceInstance']))
    
    f.write("0")

_____________
___________

# UNIT

Source: Analog.csv


Record (0): ORDER Padded with incremental index numbering


Name (0) : Analog['EuText  '] : RG 12/29: Delete repetitions and shift records up to avoid having empty spaces. 



In [120]:

analog_file = "Analog.csv"
df_analog = pd.read_csv(analog_file)


df_unit = pd.DataFrame()
df_unit['name'] = df_analog['EuText  '].drop_duplicates(keep='first').reset_index(drop=True)
df_unit = df_unit[df_unit['name'].notna() & (df_unit['name'].str.strip() != '')]


df_unit['record'] = range(1, len(df_unit['name']) + 1)

print(df_unit.head())



      name  record
0   AMPS         1
1    MPH         2
3  WATTS         3
4   VARS         4
5  VOLTS         5


In [121]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [122]:
unit_filename = "unit_dat.dat"

In [123]:
unit_config_path = os.path.join(folder_name, unit_filename)

In [124]:
with open(unit_config_path, 'w') as f:
    f.write("*\n")
    f.write("\t10\tUNIT\t0\n")
    f.write("*\t#\tName\n")

    for index, row in df_unit.iterrows():
        f.write("\t{}\t{}\n".format(row['record'],row['name']))
    
    f.write("0")

__________
____________

# OBJETO ACCUMULATOR : Comentado. No se usará

# ACCUMULATOR

ANALOG ACCUMULATOR - REFERENCE :
Resource : Analog.csv
Type (1) = Analog ['Telem_B '] : If Telem_B = 21, then T_ACCUM=1. If not defined will be set to 3: Manual
Key (3) = 8 characters.  See Key Strategy
Name (4) = Analog : Name + Desc. 
Station (5) = Analog['Name '] . pstation, XXX. Where pstation links to 'Name' in COSERV_STATION. If not defined will be set to dummy station
pAORGroup (10) = Analog['Zones '] =  Will be set to 1 unless defined in mapping document

pUNIT (19) = Analog['EUText ']  = RG 12/29: Pointer to the matching record of the object UNIT

pScale (38) = Analog['Eu_Hi ']. If not defined (Type 1 only) will be set to Scale=1, Offset=0

pALARM_GROUP (42) = Will be set to 1 unless defined in mapping document

In [125]:
comment = """ #Data CSV Name entry
analog_file = "Analog.csv"
df_analog = pd.read_csv(analog_file)

df_accumulator = pd.DataFrame()

df_accumulator['Type'] = df_analog['Telem_B  '].apply(lambda x: 1 if x == '21  ' else 3)

df_accumulator['Name'] = df_analog['Desc  ']

df_accumulator['pAORGroup'] = 1 
"""

In [126]:
comment = """
unit_mapping = df_unit.set_index('name')['record'].to_dict()
df_accumulator['pUNIT'] = df_analog['EuText  '].map(unit_mapping)

print(df_accumulator.head())
df_accumulator['pUNIT'] = df_accumulator['pUNIT'].apply(lambda x: '' if pd.isna(x) else str(int(x)))
df_accumulator['pScale'] = df_analog['EU_Hi  ']
df_accumulator['pALARM_GROUP'] = 1
"""

In [127]:
comment = """
with open('Accumulator.dat', 'w') as f:
    f.write("*\n")
    f.write("\t6\tAccumulator\t1\t4\t10\t19\t38\t42\n")
    f.write("*\tType\tName\tpAORGROUP\tpUNIT\tpScale\tpALARM_GROUP\n")

    for index, row in df_accumulator.iterrows():
        f.write("\t{}\t{}\t{}\t{}\t{}\t{}\n".format(row['Type'],row['Name'],row['pAORGroup'],row['pUNIT'],row['pScale'],row['pALARM_GROUP']))
    
    f.write("0")
    """

# RG 11/3 Object: Scale(9)

# Debe existir un archivo all_stations_equivalency.csv para operar

source: all_station_equivalency.csv

Scale(0) : Column 'Scale Factor' : RG 3/11: Delete repetitions and shift records up to avoid having empty spaces. If not defined will be set to 1

Offset(1) : If not defined will be set to 0

Name(12) : Column 'Scale Factor' : RG 3/11: Delete repetitions and shift records up to avoid having empty spaces. If not defined will be set to "<Scale #>, <Offset #>



ADVERTENCIA. LOS ARCHIVOS XLSX TIENEN UN HEADER COMPUESTO POR DOS FILAS DISTINTAS. ES RECOMENDABLE GENERAR MANUALMENTE LOS COLUMN NAMES

Point ID,Station,Pnuemonic,Description,COM,RTU,Pg,Status P,Analog P,Relay P,Value,Scale,Type,Helper Column,IP Address,DNPAddress,DNP Point,,

In [128]:
import pandas as pd
df = pd.read_csv('all_stations_equivalency.csv')

df.dropna(subset=['Scale'], inplace=True)



In [129]:
df

,Point ID,Station,Pnuemonic,Description,COM,RTU,Pg,Status P,Analog P,Relay P,Value,Scale,Type,Helper Column,IP Address,DNPAddress,DNP Point,Unnamed: 17,Unnamed: 18
40,6914,DB,RAV1,DB TX1 REG A VOLTS,3,7,NaN,NaN,0.0,NaN,NaN,0.1,ANALOG,DBRAV1ANALOG,172.17.4.81,1,24.0,NaN,NaN
41,6910,DB,P1,DB PULSE ACCUMULATOR 1,3,7,NaN,NaN,NaN,NaN,NaN,4.8,PULSE,DBP1PULSE,NaN,1,NaN,NaN,NaN
47,32662,DB,MET1SC,DB TX1 735 METER FAIL,3,7,1,6.0,NaN,NaN,NaN,REV 4MAT,STATUS,DBMET1SCSTATUS,172.17.4.81,1,176.0,NaN,NaN
56,6915,DB,RBV1,DB TX1 REG B VOLTS,3,7,NaN,NaN,1.0,NaN,NaN,0.1,ANALOG,DBRBV1ANALOG,172.17.4.81,1,25.0,NaN,NaN
57,6911,DB,P2,DB PULSE ACCUMULATOR 2,3,7,NaN,NaN,NaN,NaN,NaN,4.8,PULSE,DBP2PULSE,NaN,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,9370,OP,T1HST2,OP TX1 WINDING 2 TEMP,4,23,NaN,NaN,80.0,NaN,NaN,1,ANALOG,OPT1HST2ANALOG,172.18.3.81,1,39.0,NaN,NaN
1092,9371,OP,T1HST3,OP TX1 WINDING 3 TEMP,4,23,NaN,NaN,81.0,NaN,NaN,1,ANALOG,OPT1HST3ANALOG,172.18.3.81,1,40.0,NaN,NaN
1093,9372,OP,TX1TMP,OP TX1 TANK OIL TEMP,4,23,NaN,NaN,82.0,NaN,NaN,1,ANALOG,OPTX1TMPANALOG,172.18.3.81,1,41.0,NaN,NaN
1094,9373,OP,TC1TMP,OP LTC1 OIL TEMP,4,23,NaN,NaN,83.0,NaN,NaN,1,ANALOG,OPTC1TMPANALOG,172.18.3.81,1,42.0,NaN,NaN


In [130]:

df.drop_duplicates(subset=['Scale'], keep='first', inplace=True)


In [131]:
def prepend_if_needed(x):
    if x.startswith('.'):
        return '0' + x
    else:
        return x

In [132]:
df['Scale'] = df['Scale'].apply(prepend_if_needed)

In [133]:
df = df[~df['Scale'].str.startswith('REV')] 

In [134]:
df

,Point ID,Station,Pnuemonic,Description,COM,RTU,Pg,Status P,Analog P,Relay P,Value,Scale,Type,Helper Column,IP Address,DNPAddress,DNP Point,Unnamed: 17,Unnamed: 18
40,6914,DB,RAV1,DB TX1 REG A VOLTS,3,7,NaN,NaN,0.0,NaN,NaN,0.1,ANALOG,DBRAV1ANALOG,172.17.4.81,1,24.0,NaN,NaN
41,6910,DB,P1,DB PULSE ACCUMULATOR 1,3,7,NaN,NaN,NaN,NaN,NaN,4.8,PULSE,DBP1PULSE,NaN,1,NaN,NaN,NaN
114,6917,DB,TX1A,DB TX1 LS A AMPS,3,7,NaN,NaN,3.0,NaN,NaN,0.8,ANALOG,DBTX1AANALOG,172.17.4.81,1,27.0,NaN,NaN
188,6920,DB,TX1W,DB TX1 WATTS,3,7,NaN,NaN,6.0,NaN,NaN,32,ANALOG,DBTX1WANALOG,172.17.4.81,1,31.0,NaN,NaN
277,6924,DB,LTC1,DB LTC1 TAP POS,3,7,NaN,NaN,10.0,NaN,NaN,0.0078,ANALOG,DBLTC1ANALOG,172.17.4.81,1,21.0,NaN,NaN
295,6925,DB,1A,DB BKR 1 A AMPS,3,7,NaN,NaN,11.0,NaN,NaN,0.586223,ANALOG,DB1AANALOG,172.17.4.81,1,46.0,NaN,NaN
375,6934,DB,DCV,DC VOLTS,3,7,NaN,NaN,20.0,NaN,NaN,0.073278,ANALOG,DBDCVANALOG,172.17.4.81,1,70.0,NaN,NaN
384,7017,DB,4A,DB BKR 4 A AMPS,3,7,NaN,NaN,32.0,NaN,NaN,0.586227,ANALOG,DB4AANALOG,172.17.4.82,1,46.0,NaN,NaN
385,7018,DB,4B,DB BKR 4 B AMPS,3,7,NaN,NaN,33.0,NaN,NaN,0.586224,ANALOG,DB4BANALOG,172.17.4.82,1,47.0,NaN,NaN
401,7068,DB,7A,DB BKR 7 A AMPS,3,7,NaN,NaN,52.0,NaN,NaN,0.5862237,ANALOG,DB7AANALOG,172.17.4.83,1,46.0,NaN,NaN


salvar el objeto Scale en formato dat

In [135]:
folder_name = "SCADA_DAT_FILES"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [136]:
scale_filename = "scale_dat.dat"

In [137]:
scale_path = os.path.join(folder_name, scale_filename)

In [138]:
n = 1
with open(scale_path, 'w') as f:
    f.write("*\n")
    f.write("\t9\tScale\t0\t12\n")
    f.write("*\tScale\tName\n")

    for index, row in df.iterrows():
        f.write("\t{}\t{}\t{}\n".format(n,row['Scale'],row['Scale']))

        n = n + 1
    f.write(f"\t{len(df) + 1}\t1.0\tOne_Unit\n")
    
    f.write("0")